# Finance Knowledge Graph creation


In [1]:
# imports
import pandas as pd
from rdflib import Graph, URIRef, Namespace, Literal
from rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO, RDFS, OWL
from urllib.parse import quote
import numpy as np

In [2]:
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("../yfinance.cache"),
)

In [3]:
g = Graph()

# namespaces
FINANCE = Namespace("https://w3id.org/finance/")

COMPANY = Namespace("https://finance.yahoo.com/quote/")
SECTOR = Namespace("https://finance.yahoo.com/sectors/")

ETF = Namespace("https://w3id.org/finance/etf/")
# https://www.justetf.com/en/etf-profile.html?isin=

PERSON = Namespace("https://schema.org/person/")  

# bind prefixes
g.bind("finance", FINANCE)
g.bind("company", COMPANY)
g.bind("sector", SECTOR)
#g.bind("industry", INDUSTRY)
g.bind("etf", ETF)
g.bind("person", PERSON)

g.bind("earningsdate", FINANCE.EarningDate)
g.bind("recommendation", FINANCE.Recommendation)

# Binding

## bind classes

In [5]:
# bind classes
g.add((FINANCE.Person, RDF.type, RDFS.Class))
g.add((FINANCE.ETF, RDF.type, RDFS.Class))
g.add((FINANCE.Company, RDF.type, RDFS.Class))
g.add((FINANCE.Industry, RDF.type, RDFS.Class))
g.add((FINANCE.Sector, RDF.type, RDFS.Class))
g.add((FINANCE.Address, RDF.type, RDFS.Class))
g.add((FINANCE.ContactInformation, RDF.type, RDFS.Class))
g.add((FINANCE.EarningsDate, RDF.type, RDFS.Class))
g.add((FINANCE.Recommendation, RDF.type, RDFS.Class))

<Graph identifier=N3a1a25b0867e4e04a18b23cf7054181d (<class 'rdflib.graph.Graph'>)>

# bind properties
## ETF

In [6]:
# bind properties

# etf
## name
g.add((FINANCE.hasName, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasName, RDFS.domain, FINANCE.ETF))
g.add((FINANCE.hasName, RDFS.range, XSD.string))

g.add((FINANCE.hasTicker, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasTicker, RDFS.domain, FINANCE.ETF))
g.add((FINANCE.hasTicker, RDFS.range, XSD.string))

g.add((FINANCE.hasCurrency, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasCurrency, RDFS.domain, FINANCE.ETF))
g.add((FINANCE.hasCurrency, RDFS.range, XSD.string))

# company
g.add((FINANCE.hasCompany, RDF.type, OWL.ObjectProperty))
g.add((FINANCE.hasCompany, RDFS.domain, FINANCE.ETF))
g.add((FINANCE.hasCompany, RDFS.range, FINANCE.Company))

<Graph identifier=N3a1a25b0867e4e04a18b23cf7054181d (<class 'rdflib.graph.Graph'>)>

## company

In [7]:
# company
## name
g.add((FINANCE.hasName, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasName, RDFS.domain, FINANCE.Company))
g.add((FINANCE.hasName, RDFS.range, XSD.string))

## symbol
g.add((FINANCE.hasTicker, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasTicker, RDFS.domain, FINANCE.Company))
g.add((FINANCE.hasTicker, RDFS.range, XSD.string))

## employee
g.add((FINANCE.hasKeyExecutive, RDF.type, OWL.ObjectProperty))
g.add((FINANCE.hasKeyExecutive, RDFS.domain, FINANCE.Company))
g.add((FINANCE.hasKeyExecutive, RDFS.range, FINANCE.Person))

## url or sameAs website for the company
g.add((FINANCE.hasWebsite, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasWebsite, RDFS.domain, FINANCE.Company))
g.add((FINANCE.hasWebsite, RDFS.range, XSD.string))

## full time employees
g.add((FINANCE.hasFullTimeEmployees, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasFullTimeEmployees, RDFS.domain, FINANCE.Company))
g.add((FINANCE.hasFullTimeEmployees, RDFS.range, XSD.integer))

## industry
g.add((FINANCE.hasIndustry, RDF.type, OWL.ObjectProperty))
g.add((FINANCE.hasIndustry, RDFS.domain, FINANCE.Company))
g.add((FINANCE.hasIndustry, RDFS.range, FINANCE.Industry))

## earnings date
g.add((FINANCE.hasEarningsDate, RDF.type, OWL.ObjectProperty))
g.add((FINANCE.hasEarningsDate, RDFS.domain, FINANCE.Company))
g.add((FINANCE.hasEarningsDate, RDFS.range, FINANCE.EarningsDate))

# recommendation
g.add((FINANCE.hasRecommendation, RDF.type, OWL.ObjectProperty))
g.add((FINANCE.hasRecommendation, RDFS.domain, FINANCE.Company))
g.add((FINANCE.hasRecommendation, RDFS.range, FINANCE.Recommendation))

# address
g.add((FINANCE.hasAddress, RDF.type, OWL.ObjectProperty))
g.add((FINANCE.hasAddress, RDFS.domain, FINANCE.Company))
g.add((FINANCE.hasAddress, RDFS.range, FINANCE.Address))

# currency
g.add((FINANCE.hasCurrency, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasCurrency, RDFS.domain, FINANCE.Company))
g.add((FINANCE.hasCurrency, RDFS.range, XSD.string))

<Graph identifier=N3a1a25b0867e4e04a18b23cf7054181d (<class 'rdflib.graph.Graph'>)>

## earning date

In [8]:
# earnings date has  ['EPS Estimate', 'Reported EPS', 'Surprise(%)']
# EPS Estimate
g.add((FINANCE.hasEstimatedEPS, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasEstimatedEPS, RDFS.domain, FINANCE.EarningsDate))
g.add((FINANCE.hasEstimatedEPS, RDFS.range, XSD.float))

# Reported EPS
g.add((FINANCE.hasReportedEPS, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasReportedEPS, RDFS.domain, FINANCE.EarningsDate))
g.add((FINANCE.hasReportedEPS, RDFS.range, XSD.float))

# Surprise(%)
g.add((FINANCE.hasSurprise, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasSurprise, RDFS.domain, FINANCE.EarningsDate))
g.add((FINANCE.hasSurprise, RDFS.range, XSD.float))

# date
g.add((FINANCE.hasDate, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasDate, RDFS.domain, FINANCE.EarningsDate))
g.add((FINANCE.hasDate, RDFS.range, XSD.date))

<Graph identifier=N3a1a25b0867e4e04a18b23cf7054181d (<class 'rdflib.graph.Graph'>)>

## recommendation

In [9]:
# recommendation

# date
g.add((FINANCE.hasDate, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasDate, RDFS.domain, FINANCE.Recommendation))
g.add((FINANCE.hasDate, RDFS.range, XSD.string))

# strongbuycount
g.add((FINANCE.hasStrongBuyCount, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasStrongBuyCount, RDFS.domain, FINANCE.Recommendation))
g.add((FINANCE.hasStrongBuyCount, RDFS.range, XSD.integer))

# buycount
g.add((FINANCE.hasBuyCount, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasBuyCount, RDFS.domain, FINANCE.Recommendation))
g.add((FINANCE.hasBuyCount, RDFS.range, XSD.integer))

# holdcount
g.add((FINANCE.hasHoldCount, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasHoldCount, RDFS.domain, FINANCE.Recommendation))
g.add((FINANCE.hasHoldCount, RDFS.range, XSD.integer))

# sellcount
g.add((FINANCE.hasSellCount, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasSellCount, RDFS.domain, FINANCE.Recommendation))
g.add((FINANCE.hasSellCount, RDFS.range, XSD.integer))

# strongsellcount
g.add((FINANCE.hasStrongSellCount, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasStrongSellCount, RDFS.domain, FINANCE.Recommendation))
g.add((FINANCE.hasStrongSellCount, RDFS.range, XSD.integer))

<Graph identifier=N3a1a25b0867e4e04a18b23cf7054181d (<class 'rdflib.graph.Graph'>)>

## industry

In [13]:
# industry
## name
g.add((FINANCE.hasName, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasName, RDFS.domain, FINANCE.Industry))
g.add((FINANCE.hasName, RDFS.range, XSD.string))

<Graph identifier=N3a1a25b0867e4e04a18b23cf7054181d (<class 'rdflib.graph.Graph'>)>

## sector

In [14]:
# industry to sector
g.add((FINANCE.belongsToSector, RDF.type, OWL.ObjectProperty))
g.add((FINANCE.belongsToSector, RDFS.domain, FINANCE.Industry))
g.add((FINANCE.belongsToSector, RDFS.range, FINANCE.Sector))

<Graph identifier=N3a1a25b0867e4e04a18b23cf7054181d (<class 'rdflib.graph.Graph'>)>

## person

In [15]:
# person
## name
g.add((FINANCE.hasName, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasName, RDFS.domain, FINANCE.Person))
g.add((FINANCE.hasName, RDFS.range, XSD.string))

## gender
g.add((FINANCE.hasGender, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasGender, RDFS.domain, FINANCE.Person))
g.add((FINANCE.hasGender, RDFS.range, XSD.string))

## job title
g.add((FINANCE.hasJobTitle, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasJobTitle, RDFS.domain, FINANCE.Person))
g.add((FINANCE.hasJobTitle, RDFS.range, XSD.string))

## age
g.add((FINANCE.hasAge, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasAge, RDFS.domain, FINANCE.Person))
g.add((FINANCE.hasAge, RDFS.range, XSD.integer))

## yearborn
g.add((FINANCE.hasYearBorn, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasYearBorn, RDFS.domain, FINANCE.Person))
g.add((FINANCE.hasYearBorn, RDFS.range, XSD.integer))

## totalPay
g.add((FINANCE.hasTotalPay, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasTotalPay, RDFS.domain, FINANCE.Person))
g.add((FINANCE.hasTotalPay, RDFS.range, XSD.float))

## exercisedValue
g.add((FINANCE.hasExercisedValue, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasExercisedValue, RDFS.domain, FINANCE.Person))
g.add((FINANCE.hasExercisedValue, RDFS.range, XSD.float))

## unexercisedValue
g.add((FINANCE.hasUnexercisedValue, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasUnexercisedValue, RDFS.domain, FINANCE.Person))
g.add((FINANCE.hasUnexercisedValue, RDFS.range, XSD.float))

<Graph identifier=N3a1a25b0867e4e04a18b23cf7054181d (<class 'rdflib.graph.Graph'>)>

## address

In [16]:
# address
## street
g.add((FINANCE.hasStreet, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasStreet, RDFS.domain, FINANCE.Address))
g.add((FINANCE.hasStreet, RDFS.range, XSD.string))

## city
g.add((FINANCE.hasCity, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasCity, RDFS.domain, FINANCE.Address))
g.add((FINANCE.hasCity, RDFS.range, XSD.string))

## postal code
g.add((FINANCE.hasPostalCode, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasPostalCode, RDFS.domain, FINANCE.Address))
g.add((FINANCE.hasPostalCode, RDFS.range, XSD.string))

## country
g.add((FINANCE.hasCountry, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasCountry, RDFS.domain, FINANCE.Address))
g.add((FINANCE.hasCountry, RDFS.range, XSD.string))

## state
g.add((FINANCE.hasState, RDF.type, OWL.DatatypeProperty))
g.add((FINANCE.hasState, RDFS.domain, FINANCE.Address))
g.add((FINANCE.hasState, RDFS.range, XSD.string))

<Graph identifier=N3a1a25b0867e4e04a18b23cf7054181d (<class 'rdflib.graph.Graph'>)>

In [18]:
# safe ontology
g.serialize(destination='../data/ontology.ttl', format='turtle')

<Graph identifier=N3a1a25b0867e4e04a18b23cf7054181d (<class 'rdflib.graph.Graph'>)>

# insert data

In [19]:
import yfinance as yf
import pandas as pd
import regex as re

In [20]:
def prep_uri(namespace: Namespace, clazz: str):
    """
    This function prepares a URI for a given class. It removes special chars like & and -.
    After that it removes any amount of whitespaces and replaces them with a dash.
    :param namespace: 
    :param clazz: 
    :return: 
    """
    return namespace[quote(re.sub(r'\s+', '-', clazz.replace('&', '').replace('-', '').replace('.', '').lower()))]

## industry and sector data

In [22]:
# read industry data
sectors_industries_df = pd.read_excel('../data/sectors-industries.xlsx')
sectors_industries_df.head()

,Technology,Financial Services,Healthcare,Consumer Cyclical,Industrials,Communication Services,Consumer Defensive,Energy,Basic Materials,Real Estate,Utilities
0,Software - Infrastructure,Banks - Diversified,Drug Manufacturers - General,Internet Retail,Specialty Industrial Machinery,Internet Content & Information,Discount Stores,Oil & Gas Integrated,Specialty Chemicals,REIT - Specialty,Utilities - Regulated Electric
1,Semiconductors,Credit Services,Healthcare Plans,Auto Manufacturers,Aerospace & Defense,Telecom Services,Beverages - Non-Alcoholic,Oil & Gas E&P,Gold,REIT - Industrial,Utilities - Renewable
2,Consumer Electronics,Asset Management,Medical Devices,Restaurants,Railroads,Entertainment,Household & Personal Products,Oil & Gas Midstream,Building Materials,REIT - Retail,Utilities - Diversified
3,Software - Application,Banks - Regional,Biotechnology,Home Improvement Retail,Farm & Heavy Construction Machinery,Electronic Gaming & Multimedia,Packaged Foods,Oil & Gas Refining & Marketing,Copper,REIT - Residential,Utilities - Regulated Gas
4,Information Technology Services,Capital Markets,Diagnostics & Research,Travel Services,Building Products & Equipment,Advertising Agencies,Tobacco,Oil & Gas Equipment & Services,Steel,Real Estate Services,Utilities - Regulated Water


In [23]:
# create industry and sector triples
industry_uri_df = pd.DataFrame(columns=['industry', 'uri'])

for sector in sectors_industries_df.columns:

    sector_uri = prep_uri(SECTOR, sector)

    g.add((sector_uri, RDF.type, FINANCE.Sector))
    g.add((sector_uri, RDFS.label, Literal(sector, datatype=XSD.string)))

    for industry in sectors_industries_df[sector]:
        if pd.isna(industry):
            continue

        industry_uri = prep_uri(Namespace(sector_uri + '/'), industry)
        # add industry uri to dataframe
        row = pd.Series({'industry': industry, 'uri': industry_uri})
        industry_uri_df = pd.concat([industry_uri_df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

        g.add((industry_uri, RDF.type, FINANCE.Industry))
        g.add((industry_uri, RDFS.label, Literal(industry, datatype=XSD.string)))
        g.add((industry_uri, FINANCE.belongsToSector, sector_uri))

In [24]:
industry_uri_df

,industry,uri
0,Software - Infrastructure,https://finance.yahoo.com/sectors/technology/s...
1,Semiconductors,https://finance.yahoo.com/sectors/technology/s...
2,Consumer Electronics,https://finance.yahoo.com/sectors/technology/c...
3,Software - Application,https://finance.yahoo.com/sectors/technology/s...
4,Information Technology Services,https://finance.yahoo.com/sectors/technology/i...
...,...,...
140,Utilities - Renewable,https://finance.yahoo.com/sectors/utilities/ut...
141,Utilities - Diversified,https://finance.yahoo.com/sectors/utilities/ut...
142,Utilities - Regulated Gas,https://finance.yahoo.com/sectors/utilities/ut...
143,Utilities - Regulated Water,https://finance.yahoo.com/sectors/utilities/ut...


## company data

In [25]:
def is_valid_ticker(ticker: yf):
    """
    This function checks if a given ticker symbol is valid.
    :param ticker: 
    :return: boolean
    """
    try:
        if str(ticker.get_info()) == "{'trailingPegRatio': None}":
            return False
        return True
    except Exception as e:
        #print(e)
        return False

In [26]:
def grep_ticker_data(tickers: pd.Series, etf_uri: str):
    """
    This function accumulates relevant stock data from yahoo finance with the corresponding ticker symbol.
    :param tickers: column of ticker symbols
    :return: TODO
    """
    i = 0  # TODO remove
    invalid_tickers = []

    # get possible info data
    tickers_metadata_df = pd.DataFrame()  
    for ticker_value in tickers:
        ticker = yf.Ticker(str(ticker_value), session=session)

        if not is_valid_ticker(ticker):
            # handling ticker like 'BRK/B', 'BF/B', 'HEI/A', 'UHAL/B'
            if '/' in ticker_value:
                ticker = yf.Ticker(str(ticker_value.replace('/', '-')), session=session)
                if not is_valid_ticker(ticker):
                    invalid_tickers.append(ticker_value)
                    continue
            else:
                invalid_tickers.append(ticker_value)
                continue
        handle_info_data(ticker, etf_uri)

    print(f"a total of {len(invalid_tickers)} invalid tickers. \n"
          f"{invalid_tickers}")

    return tickers_metadata_df

In [27]:
def handle_info_data(ticker: yf.Ticker, etf_uri: str):
    """
    This function handles the info data of a given ticker.
    :param ticker: 
    :return: 
    """
    info = ticker.get_info()
    # if shortname is not available, skip and print shortname
    if 'shortName' not in info:
        print(f"shortName not available for {ticker}")
        return
    company_uri = URIRef(COMPANY + quote(info['symbol']))
    g.add((etf_uri, FINANCE.hasCompany, company_uri))
    g.add((company_uri, RDF.type, FINANCE.Company))
    print(company_uri)
    g.add((company_uri, FINANCE.hasName, Literal(info['shortName'], datatype=XSD.string)))
    g.add((company_uri, FINANCE.hasTicker, Literal(info['symbol'], datatype=XSD.string)))
    if 'website' in info:
        g.add((company_uri, FINANCE.hasWebsite, Literal(info['website'], datatype=XSD.string)))
    if 'industry' in info:
        if info['industry'] in industry_uri_df['industry'].values:
            g.add((company_uri, FINANCE.hasIndustry, industry_uri_df[industry_uri_df['industry'] == info['industry']]['uri'].values[0]))
    if 'fullTimeEmployees' in info:
        g.add((company_uri, FINANCE.hasFullTimeEmployees, Literal(info['fullTimeEmployees'], datatype=XSD.integer)))
    if 'currency' in info:
        g.add((company_uri, FINANCE.hasCurrency, Literal(info['currency'], datatype=XSD.string)))


    # add earnings
    base_earnings_uri = company_uri + '/earnings/'
    try:
        earnings_df = ticker.earnings_dates
        if earnings_df is not None:
            for index, row in earnings_df.iterrows():
                date = index.date()
                earnings_uri = URIRef(base_earnings_uri + quote(str(date)))
                g.add((company_uri, FINANCE.hasEarningsDate, earnings_uri))
                g.add((earnings_uri, RDF.type, FINANCE.EarningDate))
                g.add((earnings_uri, FINANCE.hasDate, Literal(date, datatype=XSD.date)))
                if 'EPS Estimate' in row.keys():
                    if not pd.isna(row['EPS Estimate']):
                        g.add((earnings_uri, FINANCE.hasEstimatedEPS, Literal(row['EPS Estimate'], datatype=XSD.float)))
                if 'Reported EPS' in row.keys():
                    if not pd.isna(row['Reported EPS']):
                        g.add((earnings_uri, FINANCE.hasReportedEPS, Literal(row['Reported EPS'], datatype=XSD.float)))
                if 'Surprise(%)' in row.keys():
                    if not pd.isna(row['Surprise(%)']):
                        g.add((earnings_uri, FINANCE.hasSurprise, Literal(row['Surprise(%)'], datatype=XSD.float)))
    except KeyError as e:
        print(f"skipping earnings_dates for {str(company_uri)}")
    ## done adding earnings

    # add recommendation
    try:
        base_recommendation_uri = company_uri + '/recommendation/'
        recommendation_df = ticker.get_recommendations()
        if recommendation_df is not None:
            for index, row in recommendation_df.iterrows():
                date = index
                if 'period' in row.keys():
                    if not pd.isna(row['period']):
                        date = row['period']
                        recommendation_uri = URIRef(base_recommendation_uri + quote(str(date)))
                        g.add((company_uri, FINANCE.hasRecommendation, recommendation_uri))
                        g.add((recommendation_uri, RDF.type, FINANCE.Recommendation))
                        g.add((recommendation_uri, FINANCE.hasDate, Literal(date, datatype=XSD.string)))
                    else:
                        continue
                else:
                    continue
    
                if 'strongBuy' in row.keys():
                    if not pd.isna(row['strongBuy']):
                        g.add((recommendation_uri, FINANCE.hasStrongBuyCount, Literal(row['strongBuy'], datatype=XSD.integer)))
                if 'buy' in row.keys():
                    if not pd.isna(row['buy']):
                        g.add((recommendation_uri, FINANCE.hasBuyCount, Literal(row['buy'], datatype=XSD.integer)))
                if 'hold' in row.keys():
                    if not pd.isna(row['hold']):
                        g.add((recommendation_uri, FINANCE.hasHoldCount, Literal(row['hold'], datatype=XSD.integer)))
                if 'sell' in row.keys():
                    if not pd.isna(row['sell']):
                        g.add((recommendation_uri, FINANCE.hasSellCount, Literal(row['sell'], datatype=XSD.integer)))
                if 'strongSell' in row.keys():
                    if not pd.isna(row['strongSell']):
                        g.add((recommendation_uri, FINANCE.hasStrongSellCount, Literal(row['strongSell'], datatype=XSD.integer)))
    except Exception as e:
        print(f"skipping earnings_dates for {str(company_uri)}")
    ## done adding recommendation

    for key in info.keys():
        # company officers
        if key == 'companyOfficers':
            for employee in info['companyOfficers']:
                name = re.sub(r'Mr. |Mrs. |Ms. ', '', employee['name'])

                person_uri = prep_uri(PERSON, name.lower().replace(' ', '-'))
                g.add((person_uri, RDF.type, FINANCE.Person))

                # remove Mr. and Mrs. from name
                g.add((person_uri, FINANCE.hasName, Literal(name, datatype=XSD.string)))
                # filter gender
                if 'Mr.' in employee['name']:
                    g.add((person_uri, FINANCE.hasGender, Literal('m', datatype=XSD.string)))
                if 'Mrs.' in employee['name'] or 'Ms.' in employee['name']:
                    g.add((person_uri, FINANCE.hasGender, Literal('w', datatype=XSD.string)))
                if 'title' in employee:
                    g.add((person_uri, FINANCE.hasJobTitle, Literal(employee['title'], datatype=XSD.string)))
                if 'totalPay' in employee:
                    g.add((person_uri, FINANCE.hasTotalPay, Literal(employee['totalPay'], datatype=XSD.float)))
                if 'exercisedValue' in employee:
                    g.add((person_uri, FINANCE.hasExercisedValue,
                           Literal(employee['exercisedValue'], datatype=XSD.float)))
                if 'unexercisedValue' in employee:
                    g.add((person_uri, FINANCE.hasUnexercisedValue,
                           Literal(employee['unexercisedValue'], datatype=XSD.float)))
                if 'yearBorn' in employee:
                    g.add((person_uri, FINANCE.hasYearBorn, Literal(employee['yearBorn'], datatype=XSD.integer)))
                if 'age' in employee:
                    g.add((person_uri, FINANCE.hasAge, Literal(employee['age'], datatype=XSD.integer)))
                # add employee to company
                g.add((company_uri, FINANCE.hasKeyExecutive, person_uri))
        ## done adding company officers
        # address
        address_uri = URIRef(company_uri + '/address/')
        g.add((company_uri, FINANCE.hasAddress, address_uri))
        g.add((address_uri, RDF.type, FINANCE.Address))
        if 'address1' in info:
            g.add((address_uri, FINANCE.hasStreet, Literal(info['address1'], datatype=XSD.string)))
        if 'city' in info:
            g.add((address_uri, FINANCE.hasCity, Literal(info['city'], datatype=XSD.string)))
        if 'zip' in info:
            g.add((address_uri, FINANCE.hasPostalCode, Literal(info['zip'], datatype=XSD.string)))
        if 'country' in info:
            g.add((address_uri, FINANCE.hasCountry, Literal(info['country'], datatype=XSD.string)))
        if 'state' in info:
            g.add((address_uri, FINANCE.hasState, Literal(info['state'], datatype=XSD.string)))
        # done adding address
pass

In [29]:
# read companies in the msci world
msci_world_companies_df = pd.read_excel('../data/msci-world-updated.xlsx')
sp_500_companies_df = pd.read_excel('../data/sp-500-updated.xlsx')

In [30]:
def add_etf(ticker: str):
    """
    This function adds an ETF to the knowledge graph.
    :param ticker: 
    :return: 
    """
    etf = yf.Ticker(ticker, session=session)
    info = etf.get_info()
    etf_uri = prep_uri(ETF, info['longName'])
    g.add((etf_uri, RDF.type, FINANCE.ETF))
    g.add((etf_uri, FINANCE.hasName, Literal(info['longName'], datatype=XSD.string))
    )
    g.add((etf_uri, FINANCE.hasTicker, Literal(info['symbol'], datatype=XSD.string))
    )
    if 'currency' in info:
        g.add((etf_uri, FINANCE.hasCurrency, Literal(info['currency'], datatype=XSD.string))
        )
    return etf_uri

In [31]:
# https://www.ishares.com/us/products/239696/
msci_world_uri = add_etf('URTH')
grep_ticker_data(msci_world_companies_df['Ticker'], msci_world_uri)

https://finance.yahoo.com/quote/MSFT
https://finance.yahoo.com/quote/AAPL
https://finance.yahoo.com/quote/NVDA
https://finance.yahoo.com/quote/AMZN
https://finance.yahoo.com/quote/META
https://finance.yahoo.com/quote/GOOGL
https://finance.yahoo.com/quote/GOOG
https://finance.yahoo.com/quote/LLY
https://finance.yahoo.com/quote/AVGO
https://finance.yahoo.com/quote/TSLA
https://finance.yahoo.com/quote/JPM


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BRK/B?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BRK%2FB&crumb=RvcHzk3Ziw6


https://finance.yahoo.com/quote/BRK-B
https://finance.yahoo.com/quote/UNH
https://finance.yahoo.com/quote/V
https://finance.yahoo.com/quote/XOM
https://finance.yahoo.com/quote/MA
https://finance.yahoo.com/quote/JNJ
https://finance.yahoo.com/quote/NONOF


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/NONOF?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=NONOF&crumb=RvcHzk3Ziw6


https://finance.yahoo.com/quote/ASMLF


...

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SITIF?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=SITIF&crumb=RvcHzk3Ziw6


https://finance.yahoo.com/quote/SITIF


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SITIF?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=SITIF&crumb=RvcHzk3Ziw6


a total of 1 invalid tickers. 
['XTSLA']


""



try:
    earnings_df = ticker.earnings_dates
    earnings_df = earnings_df.fillna(0)  # Fill NaN values with 0
    earnings_df = earnings_df.dropna()
    print(earnings_df)
    if earnings_df is not None:
        for row in earnings_df.iterrows():
            print(row[0])

except KeyError as e:
    print(f"skipping earnings_dates for")

managed to reduce from around 500 invalid tickers to less than 10 because of fetching tickers from a different API.

In [32]:
# https://www.ishares.com/us/products/239726/  | TICKER IVV
sp_500_uri = add_etf('IVV')
grep_ticker_data(sp_500_companies_df['Ticker'], sp_500_uri)

https://finance.yahoo.com/quote/MSFT
https://finance.yahoo.com/quote/AAPL
https://finance.yahoo.com/quote/NVDA
https://finance.yahoo.com/quote/AMZN
https://finance.yahoo.com/quote/META
https://finance.yahoo.com/quote/GOOGL


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BRK/B?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BRK%2FB&crumb=RvcHzk3Ziw6


https://finance.yahoo.com/quote/BRK-B
https://finance.yahoo.com/quote/GOOG
https://finance.yahoo.com/quote/LLY
https://finance.yahoo.com/quote/AVGO
https://finance.yahoo.com/quote/TSLA
https://finance.yahoo.com/quote/JPM
https://finance.yahoo.com/quote/UNH
https://finance.yahoo.com/quote/V
https://finance.yahoo.com/quote/XOM
https://finance.yahoo.com/quote/MA
https://finance.yahoo.com/quote/JNJ
https://finance.yahoo.com/quote/HD
https://finance.yahoo.com/quote/PG
https://finance.yahoo.com/quote/COST
https://finance.yahoo.com/quote/MRK
https://finance.yahoo.com/quote/AMD
https://finance.yahoo.com/quote/ABBV
https://finance.yahoo.com/quote/CRM
https://finance.yahoo.com/quote/CVX
https://finance.yahoo.com/quote/NFLX
https://finance.yahoo.com/quote/ADBE
https://finance.yahoo.com/quote/WMT
https://finance.yahoo.com/quote/BAC
https://finance.yahoo.com/quote/ACN
https://finance.yahoo.com/quote/KO
https://finance.yahoo.com/quote/PEP
https://finance.yahoo.com/quote/TMO
https://finance.yahoo.com

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=RvcHzk3Ziw6


https://finance.yahoo.com/quote/ZTS
https://finance.yahoo.com/quote/BX
https://finance.yahoo.com/quote/GILD
https://finance.yahoo.com/quote/FI
https://finance.yahoo.com/quote/SNPS
https://finance.yahoo.com/quote/EQIX
https://finance.yahoo.com/quote/TMUS
https://finance.yahoo.com/quote/CDNS
https://finance.yahoo.com/quote/CME
https://finance.yahoo.com/quote/ICE
https://finance.yahoo.com/quote/SHW
https://finance.yahoo.com/quote/WM
https://finance.yahoo.com/quote/CSX
https://finance.yahoo.com/quote/CMG
https://finance.yahoo.com/quote/SO
https://finance.yahoo.com/quote/MO
https://finance.yahoo.com/quote/ITW
https://finance.yahoo.com/quote/CL
https://finance.yahoo.com/quote/DUK
https://finance.yahoo.com/quote/TGT
https://finance.yahoo.com/quote/ANET
https://finance.yahoo.com/quote/MCK
https://finance.yahoo.com/quote/SLB
https://finance.yahoo.com/quote/PH
https://finance.yahoo.com/quote/ABNB
https://finance.yahoo.com/quote/BDX
https://finance.yahoo.com/quote/EOG
https://finance.yahoo.com/qu

...

NAN: No earnings dates found, symbol may be delisted
NWS: No earnings dates found, symbol may be delisted


https://finance.yahoo.com/quote/NWS


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/NWS?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=NWS&crumb=RvcHzk3Ziw6


a total of 1 invalid tickers. 
['XTSLA']


""


In [34]:
# safe kg
g.serialize(destination='../data/finance-kg.ttl', format='turtle')

<Graph identifier=N3a1a25b0867e4e04a18b23cf7054181d (<class 'rdflib.graph.Graph'>)>